In [ ]:
library(bnlearn)
library(Rgraphviz)
library(parallel)
library(tidyverse)
suppressWarnings(library(dplyr))
no_cores <- detectCores()-1
cl <- makeCluster(no_cores)
### dataset needs to be preprocessed because  for example had values as factors. My functions does not work w/factorslearning.test

Today i learnt that the k2 algorithm infers the dag, the directed acyclic graph, so the structure of the bayesian network from the dataset. Not from the fitted model. 
What i initially thought was that you happen to have a model and you want to reverse engineer that. 

In [134]:
df <- data.frame(x1=c(1,1,0,1,0,0,1,0,1,0), x2=c(0,1,0,1,0,1,1,0,1,0), x3=c(0,1,1,1,0,1,1,0,1,0))

In [139]:
node.values <- function(dataset, parents) {
    # retrieves the unique values of the nodes
    #return a list of vectors, 
    # each vector contains the unique values of the corresponding node
    output <- list()
    for (kk in seq_along(parents)) {
        node <- parents[kk]
        node.vals <- unique(dataset[[node]]) #it was levels()
        output[[kk]] <- node.vals
    }
    return(output)
}

old.alphaijk <- function(i, pii, dataset, nodes.order) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(dataset, all.nodes))
    output <- c()
    for (tracker in 1:dim(unique.instantiations)[1]){
        aa <- 0
        for (j in 1:dim(subset)[1]) {
            ifelse(all(as.vector(unlist(subset[j,])) == as.vector(unlist(unique.instantiations[tracker,]))), aa <- aa +1, FALSE)
        }
        output <- c(output,aa)
    }
    return(output)
}

## OPtimized by copilot:
fastalphaijk <- function(i, pii, dataset, nodes.order) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(dataset, all.nodes))
    colnames(unique.instantiations) <- colnames(subset)
    subset.counted <- subset |> group_by(across(everything())) |> summarise(n = n(), .groups="drop")
    checking <-  left_join(unique.instantiations, subset.counted, by=colnames(subset), na_matches="na") 
    checking[is.na(checking)]<- 0
    return(checking$n)
}


k2helper <- function(i, pii, dataset, nodes.order) {
    # the "f" function in the pdf
    phi_i <- expand.grid(node.values(dataset, pii))
    #cat("phi.i", phi_i)
    q_i <- dim(phi_i)[1]
    #cat("qi.i", q_i)
    nodes <- colnames(dataset)
    #cat("nodes", nodes)
    i.th.node <- nodes[i]
    #cat("ithnode", i.th.node)
    v_i <- node.values(dataset, i.th.node)
    #cat("vi", c(v_i))
    r_i <- length(v_i[[1]])
    #cat("ri", r_i)
    aijk <- prod(factorial(fastalphaijk(i, pii, dataset, nodes.order)))
    numeratore <- factorial(r_i-1)
    nij <- fastalphaijk(0, pii, dataset, nodes.order) # i = 0 deactivates actual node
    nij <- nij + r_i-1
    denominatore <- factorial(nij)
    frazione <- numeratore/denominatore
    #print(aijk)
    #print(numeratore)
    #print(frazione)
    return(prod(frazione)* aijk)
}

k2 <- function(dataset, nodes.order, upperbound) {
    nodes <- colnames(dataset)
    for (i in seq_along(nodes.order)) {
        actual_node <- nodes.order[i]
        pii <- c()
        p.old <- k2helper(i, pii,dataset, nodes.order)
        proceed <- TRUE
        roof <- 0
        while (proceed && length(pii) < upperbound) {
            predecessors_idx <- ifelse(i-1>=0, i-1, 0)
            z <- nodes.order[0:predecessors_idx]
            piiuz <- unique(pii,z)
            p.new <- k2helper(i, piiuz, dataset, nodes.order)
            roof <- roof+1
            if (p.new > p.old) {
                p.old <- p.new
                pii <- piiuz
            } else  {
                proceed <- FALSE
            }
        }
        cat('\nNode', nodes.order[i], '--> Parents:', pii,'\n') 
    }
    return(TRUE)
}

In [144]:
k2(dataset=df, upperbound=3, nodes.order=colnames(df))


Node x1 --> Parents: 

Node x2 --> Parents: 

Node x3 --> Parents: 


[1] TRUE

In [137]:
fastalphaijk(0, c("x1","x2"), df, c("x1","x2", "x3"))

[1] 1 4 4 1

In [138]:
unique(c(1,2,3,4), c(3,4,4,4))

[1] 1 2 3 4

[1] "c(2, 2, 1, 1, 1, 3, 3, 2, 2, 2, 2, 3, 3, 3, 2, 1, 1, 3, 2, 2, 3, 3, 3, 2, 1, 2, 1, 3, 3, 3, 1, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3, 1, 1, 3, 1, 2, 1, 3, 1, 3, 2, 3, 1, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 2, 1, 1, 3, 3, 2, 2, 3, 3, 2, 1, 3, 2, 3, 2, 1, 3, 3, 2, 1, 2, 3, 3, 1, 3, 1, 2, 2, 2, 3, 2, 1, 1, 3, 3, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 2, 3, 2, 1, 2, 2, 2, 1, 1, 2, 3, 2, 2, 1, 1, 2, 3, 2, 3, 3, 2, 3, 3, 1, 1, 3, 2, 3, 3, 1, 2, 1, 3, 3, 1, 2, 3, 2, 2, 3, 3, 1, 2, 2, 2, 3, 1, 3, 1, 1, 3, \n2, 1, 3, 1, 2, 3, 1, 2, 3, 1, 2, 1, 3, 2, 1, 3, 3, 1, 3, 2, 3, 3, 1, 3, 3, 1, 2, 3, 3, 2, 1, 3, 2, 1, 2, 3, 1, 1, 3, 1, 2, 1, 3, 2, 2, 1, 3, 3, 1, 3, 2, 3, 3, 3, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 1, 2, 1, 2, 2, 3, 2, 1, 2, 2, 2, 3, 2, 2, 3, 2, 3, 1, 3, 3, 1, 2, 1, 3, 2, 3, 2, 3, 2, 1, 2, 2, 3, 2, 3, 2, 2, 3, 1, 1, 3, 3, 1, 3, 2, 1, 3, 1, 1, 3, 1, 3, 3, 3, 2, 1, 2, 3, 1, 3, 1, 3, 3, 3, 2, 2, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 1, 3, 1, 2, 2, 2, 1, 2, 2, 2, 3, 1, 3, 1, 2, 2, 2, 2, 3, 1, 2, 3, 1, 2, 2, \n3, 2, 2, 2, 2, 1, 3, 1, 3, 3, 2, 1, 1, 3, 1, 3, 1, 1, 1, 2, 2, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 2, 1, 3, 2, 1, 2, 2, 2, 1, 3, 1, 3, 1, 3, 1, 1, 1, 2, 1, 1, 1, 3, 1, 2, 3, 2, 1, 3, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 3, 3, 2, 3, 2, 1, 3, 1, 1, 2, 2, 2, 2, 3, 3, 1, 3, 2, 1, 2, 2, 1, 2, 3, 3, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 3, 2, 2, 3, 1, 2, 3, 3, 1, 1, 3, 2, 3, 2, 3, 3, 3, 3, 2, 2, 2, 1, 2, 3, 1, 3, 3, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 3, 3, 2, 1, 2, 2, 1, 1, 3, 2, 1, 1, 1, 3, 3, 3, 3, 2, 1, 2, 3, 2, 1, 3, \n1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 3, 3, 2, 1, 2, 1, 1, 2, 3, 1, 2, 2, 1, 3, 1, 2, 2, 2, 1, 3, 1, 1, 1, 3, 3, 1, 2, 2, 1, 2, 2, 1, 3, 3, 1, 1, 3, 1, 1, 3, 2, 2, 2, 1, 2, 1, 3, 1, 2, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 3, 3, 1, 1, 2, 3, 2, 2, 1, 2, 3, 3, 1, 1, 3, 3, 2, 1, 1, 2, 1, 1, 3, 3, 1, 2, 3, 3, 2, 2, 1, 2, 1, 3, 1, 2, 1, 1, 1, 3, 1, 3, 3, 2, 3, 3, 2, 2, 2, 1, 3, 3, 2, 3, 2, 3, 1, 1, 3, 3, 3, 3, 2, 2, 1, 3, 3, 3, 3, 2, 2, 2, 1, 1, 3, 2, 2, 3, 2, 1, 3, 2, 2, 3, 1, 2, 2, 1, 2, 1, 3, 1, 3, 3, 1, \n1, 3, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 1, 3, 2, 1, 2, 3, 1, 2, 3, 1, 2, 2, 2, 1, 3, 2, 1, 1, 1, 3, 1, 1, 2, 1, 2, 1, 2, 1, 3, 2, 1, 1, 1, 2, 3, 1, 2, 3, 1, 3, 3, 2, 2, 2, 3, 1, 1, 2, 2, 2, 2, 1, 1, 3, 1, 3, 2, 2, 1, 2, 3, 1, 1, 2, 3, 2, 2, 3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 3, 3, 1, 1, 1, 2, 1, 1, 1, 2, 3, 2, 2, 1, 2, 2, 2, 1, 3, 3, 1, 3, 3, 1, 3, 3, 2, 3, 3, 3, 1, 3, 1, 1, 2, 3, 2, 3, 2, 3, 1, 1, 1, 3, 3, 2, 3, 2, 2, 3, 2, 1, 1, 2, 1, 1, 2, 3, 2, 3, 3, 2, 2, 1, 2, 3, 1, 2, 1, 2, 2, \n3, 3, 1, 3, 1, 1, 2, 1, 3, 2, 1, 3, 1, 3, 1, 2, 2, 3, 3, 2, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 1, 3, 1, 3, 3, 3, 3, 3, 3, 2, 1, 1, 3, 3, 3, 3, 2, 1, 2, 3, 3, 2, 1, 1, 3, 1, 1, 2, 2, 3, 2, 2, 1, 1, 3, 1, 3, 1, 2, 1, 1, 2, 2, 3, 2, 3, 3, 3, 2, 3, 2, 3, 1, 3, 3, 2, 2, 2, 1, 2, 1, 1, 3, 3, 2, 3, 2, 1, 2, 2, 3, 3, 3, 1, 2, 3, 1, 1, 3, 2, 1, 3, 3, 2, 2, 2, 1, 1, 3, 2, 3, 3, 3, 1, 2, 2, 2, 3, 2, 3, 1, 2, 1, 1, 3, 3, 3, 1, 3, 1, 2, 1, 3, 3, 3, 3, 1, 1, 3, 2, 1, 3, 2, 1, 1, 1, 2, 1, 3, 3, 3, 2, 2, 3, 2, 3, 1, \n3, 1, 2, 2, 1, 3, 1, 2, 2, 1, 1, 1, 3, 3, 2, 2, 1, 2, 3, 3, 3, 1, 3, 1, 3, 3, 2, 1, 2, 2, 3, 1, 2, 2, 3, 1, 3, 1, 1, 1, 3, 2, 3, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 3, 2, 1, 2, 1, 3, 2, 1, 1, 3, 3, 1, 1, 3, 3, 2, 2, 2, 3, 3, 3, 2, 3, 2, 3, 1, 1, 3, 3, 1, 3, 3, 1, 2, 3, 2, 3, 3, 1, 2, 2, 1, 2, 2, 3, 3, 2, 1, 1, 3, 2, 2, 2, 2, 1, 3, 1, 2, 3, 2, 2, 1, 3, 2, 2, 1, 1, 3, 1, 1, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 2, 1, 3, 1, 3, 2, 1, 1, 2, 1, 2, 3, 1, 2, 3, 1, 3, 3, 3, 3, 1, 1, 1, 1, 2, 2, 1, 1, 3, 2, \n1, 2, 1, 3, 2, 2, 3, 2, 2, 1, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 3, 2, 3, 3, 1, 2, 2, 2, 1, 2, 3, 1, 1, 2, 2, 1, 3, 3, 1, 1, 3, 1, 2, 3, 3, 3, 1, 3, 2, 1, 3, 2, 2, 1, 1, 2, 1, 3, 3, 1, 1, 1, 3, 1, 2, 3, 2, 2, 2, 3, 2, 1, 1, 2, 2, 3, 2, 3, 3, 1, 2, 3, 2, 3, 3, 1, 2, 1, 1, 2, 3, 2, 1, 1, 2, 3, 1, 3, 2, 2, 3, 2, 3, 3, 1, 1, 1, 3, 3, 2, 1, 3, 3, 2, 2, 1, 2, 1, 3, 2, 1, 3, 2, 1, 1, 1, 2, 2, 3, 3, 2, 2, 2, 1, 3, 1, 2, 2, 2, 3, 2, 2, 1, 3, 3, 1, 3, 2, 1, 1, 3, 1, 2, 1, 3, 1, 3, 1